In [1]:
!rm -rf ../mixnet_k/wide_resnet_5/poly

In [2]:
%config IPCompleter.greedy=True
import numpy as np
import tensorflow as tf
import time

In [3]:
%run ./datagen.py
datagen, (x_train, y_train), (x_test, y_test) = data_preparation()

In [4]:
save_dir = '../mixnet_k/wide_resnet_5/'
batch_size = 100
iterations = x_train.shape[0] // batch_size
epochs = 500
old_acc = 0
regularizer = tf.contrib.layers.l2_regularizer(scale=1e-4)

k = 5


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [5]:
def act(x, i):
    
    # ax^2 + bx + c
        
    if i == 1: 
        first_coef = tf.Variable(0.31486639, trainable=True, name='first_coef', dtype=tf.float32)
        second_coef = tf.Variable(0.49538696, trainable=True, name='second_coef', dtype=tf.float32)
        third_coef = tf.Variable(0.1256594, trainable=True, name='third_coef', dtype=tf.float32)
        y = tf.math.polyval([first_coef, second_coef, third_coef], x)
        
    elif i == 2:  
        first_coef = tf.Variable(0.14825453, trainable=True, name='first_coef', dtype=tf.float32)
        second_coef = tf.Variable(0.49779217, trainable=True, name='second_coef', dtype=tf.float32)
        third_coef = tf.Variable(0.26774763, trainable=True, name='third_coef', dtype=tf.float32)
        y = tf.math.polyval([first_coef, second_coef, third_coef], x)
        
    elif i == 3:  
        first_coef = tf.Variable(0.15073162, trainable=True, name='first_coef', dtype=tf.float32)
        second_coef = tf.Variable(0.48908482, trainable=True, name='second_coef', dtype=tf.float32)
        third_coef = tf.Variable(0.25940762, trainable=True, name='third_coef', dtype=tf.float32)
        y = tf.math.polyval([first_coef, second_coef, third_coef], x)

    elif i == 4:  
        first_coef = tf.Variable(0.14494386, trainable=True, name='first_coef', dtype=tf.float32)
        second_coef = tf.Variable(0.48223472, trainable=True, name='second_coef', dtype=tf.float32)
        third_coef = tf.Variable(0.26587296, trainable=True, name='third_coef', dtype=tf.float32)
        y = tf.math.polyval([first_coef, second_coef, third_coef], x)

    elif i == 5:  
        first_coef = tf.Variable(0.11254344, trainable=True, name='first_coef', dtype=tf.float32)
        second_coef = tf.Variable(0.47676477, trainable=True, name='second_coef', dtype=tf.float32)
        third_coef = tf.Variable(0.33750685, trainable=True, name='third_coef', dtype=tf.float32)
        y = tf.math.polyval([first_coef, second_coef, third_coef], x)

    else:  
        first_coef = tf.Variable(0.08870094, trainable=True, name='first_coef', dtype=tf.float32)
        second_coef = tf.Variable(0.49866869, trainable=True, name='second_coef', dtype=tf.float32)
        third_coef = tf.Variable(0.44814718, trainable=True, name='third_coef', dtype=tf.float32)
        y = tf.math.polyval([first_coef, second_coef, third_coef], x)

    return y

In [6]:
# resnet layer
def res_layer(inputs, filter_num, filter_size, stride, is_train, act_num, 
              batch_norm=True, activation=True):

    x = inputs

    if batch_norm:
        x = tf.layers.batch_normalization(x, training=is_train)
    if activation:
        x = act(x, act_num)
    x = tf.layers.conv2d(inputs=x, filters=filter_num, kernel_regularizer=regularizer, 
                         kernel_size=filter_size, strides=stride, padding='same')
        
    return x

In [7]:
def wide_resnet(inputs, k, is_train):
    
    act_num = 0

    with tf.variable_scope("1st_Conv"):
        x = tf.layers.conv2d(inputs=inputs, filters=16, kernel_regularizer=regularizer, 
                             kernel_size=3, strides=1, padding='same')
        x = tf.layers.batch_normalization(x, training=is_train)
        x = tf.nn.relu(x)
    
    x_temp_0 = x
    
    with tf.variable_scope('ResBlock_%d_%d' % (1, 1)):
        
        with tf.variable_scope('conv1'):
            x = res_layer(x, 16*k, 3, 1, is_train, act_num, 
                          batch_norm=False, activation=False)
            act_num += 1
            
        x = tf.layers.dropout(x, 0.1)
        
        with tf.variable_scope('conv2'):
            x = res_layer(x, 16*k, 3, 1, is_train, act_num)
            act_num += 1
    
        with tf.variable_scope('x_plus_shortcut'):
            shortcut = tf.layers.conv2d(inputs=x_temp_0, filters=16*k, 
                                        kernel_size=1, strides=1, padding='same')
            x = x + shortcut             
        
    x_temp_1 = x
    
    with tf.variable_scope('ResBlock_%d_%d' % (2, 1)):
        
        with tf.variable_scope('conv1'):
            x = res_layer(x, 32*k, 3, 2, is_train, act_num)
        act_num += 1
        
        x = tf.layers.dropout(x, 0.1)
                
        with tf.variable_scope('conv2'):
            x = res_layer(x, 32*k, 3, 1, is_train, act_num)
        act_num += 1
        
        with tf.variable_scope('x_plus_shortcut'):
            shortcut = tf.layers.conv2d(inputs=x_temp_0, filters=32*k, 
                                        kernel_size=1, strides=2, padding='same')
            x = x + shortcut
            
            shortcut = tf.layers.conv2d(inputs=x_temp_1, filters=32*k, 
                                        kernel_size=1, strides=2, padding='same')
            x = x + shortcut
        
    x_temp_2 = x    
    
    with tf.variable_scope('ResBlock_%d_%d' % (3, 1)):
        
        with tf.variable_scope('conv1'):
            x = res_layer(x, 64*k, 3, 2, is_train, act_num)
            act_num += 1
            
        x = tf.layers.dropout(x, 0.1)
                
        with tf.variable_scope('conv2'):
            x = res_layer(x, 64*k, 3, 1, is_train, act_num)
            act_num += 1
            
        with tf.variable_scope('x_plus_shortcut'):
            
            shortcut = tf.layers.conv2d(inputs=x_temp_0, filters=64*k, 
                                        kernel_size=1, strides=4, padding='same')
            x = x + shortcut

            shortcut = tf.layers.conv2d(inputs=x_temp_1, filters=64*k, 
                                        kernel_size=1, strides=4, padding='same')
            x = x + shortcut

            shortcut = tf.layers.conv2d(inputs=x_temp_2, filters=64*k, 
                                        kernel_size=1, strides=2, padding='same')
            x = x + shortcut

    with tf.variable_scope("AfterResBlock"):
        x = tf.layers.batch_normalization(x, training=is_train)
        x = act(x, act_num)
        x = tf.layers.average_pooling2d(x, pool_size=8, strides=8, padding='SAME', name='ave_pool')

    with tf.variable_scope("Flatten"):
        x = tf.transpose(x, perm=[0, 3, 1, 2])
        x = tf.layers.flatten(x)

    with tf.variable_scope("Prediction"):
        pred = tf.layers.dense(x, units=10, kernel_regularizer=regularizer)
        
    return pred

In [8]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"

tf.reset_default_graph()

with tf.device('/GPU:3'):

    inputs = tf.placeholder(tf.float32, [None, 32, 32, 3], name='input')
    outputs = tf.placeholder(tf.float32, [None, 10], name='output')
    is_train = tf.placeholder(tf.bool, name='is_train')

    global_step = tf.Variable(0, trainable=False)
    
    boundaries = [5*iterations, 10*iterations, 15*iterations]
    values = [1e-3, 5e-4, 1e-4, 1e-5]
    l_r = tf.train.piecewise_constant(global_step, boundaries, values)

    tf.summary.scalar('learning_rate', l_r)

    opt = tf.train.AdamOptimizer(learning_rate=l_r)

    pred = wide_resnet(inputs, k, is_train)
    
#     l2_loss = tf.losses.get_regularization_loss()
    loss = tf.losses.softmax_cross_entropy(outputs, pred)
    
    grads = opt.compute_gradients(loss)

    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(outputs, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

    with tf.control_dependencies(update_ops):
        train_op = opt.apply_gradients(grads, global_step=global_step)

    restore_vars = []

    for var in tf.global_variables()[1:51+18]:
        if '_coef:0' not in var.name:
            restore_vars.append(var)
    saver = tf.train.Saver(restore_vars)
    
    new_saver = tf.train.Saver(tf.global_variables(), max_to_keep=5)

#     def add_hist(train_vars):
#         for i in train_vars:
#             name = i.name.split(":")[0] + '/value'
#             value = i.value()
#             tf.summary.histogram(name, value)

#     add_hist(tf.trainable_variables())
    tf.summary.scalar('loss', loss)
    tf.summary.scalar('accuracy', accuracy)
    merged = tf.summary.merge_all()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Use keras.layers.batch_normalization instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.average_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [ ]:
config = tf.ConfigProto(allow_soft_placement=True,
                        log_device_placement=True)
config.gpu_options.allow_growth = True

sess = tf.Session(config=config)

batch_gen = datagen.flow(x_train, y_train, batch_size=batch_size)

print('*****************2nd training stage*****************')
sess.run(tf.global_variables_initializer())
saver.restore(sess, save_dir+'cifar10.ckpt-183500')
train_writer = tf.summary.FileWriter(save_dir+'poly/train', sess.graph)

for m in range(epochs):
        start = time.time()
        for i in range(iterations):
            x_batch, y_batch = next(batch_gen)
            _, loss_train, summary = sess.run([train_op, loss, merged], 
                                     {inputs: x_batch, outputs: y_batch, is_train: True})
        
            train_writer.add_summary(summary, m*iterations + i + 1)

        val_accs = []
        for i in range(5000//(batch_size*5)):
            val_acc = sess.run(accuracy, {inputs: x_test[i*batch_size*5: (i+1)*batch_size*5],
                                          outputs: y_test[i*batch_size*5: (i+1)*batch_size*5],
                                          is_train: False})
            val_accs.append(val_acc)

        if np.mean(val_accs) > old_acc:
            old_acc = np.mean(val_accs)
            new_saver.save(sess, save_dir+'poly/cifar10.ckpt', global_step=global_step)

        end = time.time()
        print('Epoch: {}'.format(m + 1),
              'Train_loss: {:.3f}'.format(loss_train),
              'Val_acc: {:.3f}'.format(np.mean(val_accs)),
              'Time consumed: {:.4f} s'.format(end - start))

print('*****************Training End!*****************')
sess.close()

*****************2nd training stage*****************
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../mixnet_k/wide_resnet_5/cifar10.ckpt-183500
Epoch: 1 Train_loss: 6.533 Val_acc: 0.346 Time consumed: 46.9670 s
Epoch: 2 Train_loss: 2.015 Val_acc: 0.629 Time consumed: 42.5119 s
Epoch: 3 Train_loss: 1.636 Val_acc: 0.732 Time consumed: 42.7098 s
Epoch: 4 Train_loss: 1.168 Val_acc: 0.772 Time consumed: 42.7662 s
Epoch: 5 Train_loss: 1.179 Val_acc: 0.790 Time consumed: 42.7067 s
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Epoch: 6 Train_loss: 0.974 Val_acc: 0.810 Time consumed: 42.2202 s
Epoch: 7 Train_loss: 0.922 Val_acc: 0.829 Time consumed: 42.2050 s
Epoch: 8 Train_loss: 1.002 Val_acc: 0.836 Time consumed: 42.3981 s
Epoch: 9 Train_loss: 0.735 Val_acc: 0.854 Time consumed: 42.6077 s
Epoch: 10 Train_loss: 0.753 Val_acc: 0.820 Time consumed: 41.9642 s
Epoch: 11 Train_l